# Crop REMA stripes by the polygon from 3 (around channel)

based on https://rasterio.readthedocs.io/en/stable/topics/masking-by-shapefile.html 
but also using tarfile

This script unzips the raster, crops it, then saves a tiff

In [1]:
import rasterio as rio
import rasterio.mask
import tarfile
import fiona

import os
import sys
#working_dir = '/Volumes/arc_02/REMOTE_SENSING/'
#working_dir = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/"
working_dir = '/Users/home/whitefar/DATA/FIELD_ANT_19/'

sys.path.append(os.path.abspath(working_dir))
sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/code/'))
#sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/'))
os.chdir(working_dir)

In [2]:
#zipped_stripes_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/test_stripe/"
zipped_stripes_path = "/Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_over_kamb_channel/"

#cropped_stripes_path = zipped_stripes_path
#cropped_stripes_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/cropped_stripes/"
cropped_stripes_path = "/Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/"


zipped_stripes_fnames = [f for f in os.listdir(zipped_stripes_path) if os.path.isfile(os.path.join(zipped_stripes_path, f))]

temp_tiffs_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/stripe_tiffs/" #save them temporararily

#zipped_stripes_fnames

In [3]:
#these filenames came from looking at the polygon file of stripes in qgis

for_mosaic = ["SETSM_WV01_20161109_1020010058134D00_10200100576C9100_seg1_2m_v1.0",
              "SETSM_WV01_20161027_1020010056BA2E00_1020010058DFD800_seg1_2m_v1.0",
             "SETSM_WV02_20161220_1030010061866000_103001006007BA00_seg1_2m_v1.0",
             "SETSM_WV03_20161220_1040010026391800_1040010026480500_seg1_2m_v1.0"]

zipped_stripes_fnames = [name + '.tar.gz' for name in for_mosaic]
zipped_stripes_fnames

['SETSM_WV01_20161109_1020010058134D00_10200100576C9100_seg1_2m_v1.0.tar.gz',
 'SETSM_WV01_20161027_1020010056BA2E00_1020010058DFD800_seg1_2m_v1.0.tar.gz',
 'SETSM_WV02_20161220_1030010061866000_103001006007BA00_seg1_2m_v1.0.tar.gz',
 'SETSM_WV03_20161220_1040010026391800_1040010026480500_seg1_2m_v1.0.tar.gz']

In [4]:
with fiona.open("/Users/home/whitefar/DATA/FIELD_ANT_19/study_area_buffer.shp", "r") as shapefile:
    field_area = [feature["geometry"] for feature in shapefile]

In [5]:
def crop_stripe(crop_to_area,zipped_stripe_fname, zipped_stripes_path, cropped_stripes_path, temp_tiffs_path):
       
    #remove .tar.gz from the name
    stripe_name = zipped_stripe_fname[:-7]
    
    zipped_stripe_path =  zipped_stripes_path + zipped_stripe_fname 
    
    #path for output tiff
    tiff_stripe_fname = stripe_name + "_dem.tif"
    
    #extract the .tar.gz and save as .tiff
    with tarfile.open(zipped_stripe_path) as tar:
        stripe_tiff = tar.extract(member=tiff_stripe_fname, path=temp_tiffs_path)
        
    #crop the .tiff    
    with rio.open(temp_tiffs_path + tiff_stripe_fname) as src:
        #print(src.transform)
        out_image, out_transform = rasterio.mask.mask(src, crop_to_area,crop=True)
        data_type = out_image.dtype
        out_meta = src.meta.copy()
        
    #print(out_transform)
    
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 "dtype" : data_type})
    
    cropped_stripe_path = cropped_stripes_path + tiff_stripe_fname
    
    #write the cropped tiff to file
    with rio.open(cropped_stripe_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    #remove the uncropped tiff
    os.remove(temp_tiffs_path + tiff_stripe_fname)
        
    print(stripe_name + "cropped and written to "+ cropped_stripe_path)

In [6]:
for i, zipped_stripe_fname in enumerate(zipped_stripes_fnames):
    
    crop_stripe(field_area, zipped_stripe_fname, zipped_stripes_path, cropped_stripes_path, temp_tiffs_path)
    
    print(f"{i+1}/{len(zipped_stripes_fnames)}")
       

SETSM_WV01_20161109_1020010058134D00_10200100576C9100_seg1_2m_v1.0cropped and written to /Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/SETSM_WV01_20161109_1020010058134D00_10200100576C9100_seg1_2m_v1.0_dem.tif
1/4
SETSM_WV01_20161027_1020010056BA2E00_1020010058DFD800_seg1_2m_v1.0cropped and written to /Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/SETSM_WV01_20161027_1020010056BA2E00_1020010058DFD800_seg1_2m_v1.0_dem.tif
2/4
SETSM_WV02_20161220_1030010061866000_103001006007BA00_seg1_2m_v1.0cropped and written to /Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/SETSM_WV02_20161220_1030010061866000_103001006007BA00_seg1_2m_v1.0_dem.tif
3/4
SETSM_WV03_20161220_1040010026391800_1040010026480500_seg1_2m_v1.0cropped and written to /Volumes/arc_02/REMOTE_SENSING/REMA/REMA_stripes_1920_fieldsite/SETSM_WV03_20161220_1040010026391800_1040010026480500_seg1_2m_v1.0_dem.tif
4/4
